# Producer Kafka Pentru Tranzactii

Vom folosi un producer pentru a trimite evenimente catre topicul din Kafka. Vom citi datele  dintr-un fisier parquet si apoi vom crea un eveniemnt random din acestea.

In [2]:
KAFKA_BOOTSTRAP_SERVER = 'bd-kafka:29092' # adresa de bootstrap a serverului kafka din Docker
TOPIC = 'bd-proiect' # numele topicului Kafka

In [3]:
import pandas as pd
import os

df=pd.read_parquet(os.path.join(os.getcwd()+"/proiect/output/mini_test_df.parquet"))
df.head()

Transaction ID                           Customer ID  \
0  bf3db41c-25a9-4e6d-b87f-13e5fabe35dd  7300dcf3-df6f-439f-ba13-a98f18b972f3   
1  326199a2-7e5c-4c76-ab15-805ce119b381  3612d33f-bacf-4337-a819-1bda24058370   
2  f77b607f-230a-4981-bcf2-5db8f26caa1f  a078ac2b-3548-480e-a244-973e116f1229   
3  f675cd27-0c21-4b63-b114-3b4454f53cb7  8bc028c7-d6b5-46ac-8e72-3d20444e908d   
4  59ac0ba0-fd98-48be-aeb1-5da4f6aee87c  b11bf3de-e5d3-4c92-8d76-b6ece6275591   

   Transaction Amount     Transaction Date Payment Method Product Category  \
0              680.17  2024-01-18 23:25:10     debit card      electronics   
1              260.69  2024-02-19 02:48:24     debit card      electronics   
2               36.12  2024-03-26 14:40:52         PayPal     toys & games   
3              438.16  2024-02-28 12:04:16    credit card      electronics   
4              112.26  2024-01-26 03:33:03     debit card    home & garden   

   Quantity  Customer Age Customer Location Device Used       IP Address  \
0         5            39       South Tracy      tablet     61.52.160.51   
1         5            15     New Aaronport      tablet   217.121.61.188   
2         2            39       East Joshua      tablet      9.65.99.208   
3         1            34      Jenniferberg      tablet     26.8.155.180   
4         4            48     New Dianebury     desktop  138.228.171.245   

                                    Shipping Address  \
0  30470 Amanda Causeway Suite 446\nLake Ashleyla...   
1               628 Cole Square\nPort Mary, VT 38674   
2         85492 Hill Green\nWest Alexander, TX 53337   
3                   PSC 8138, Box 0814\nAPO AA 42879   
4                043 Jean Falls\nMeganland, KS 98284   

                                     Billing Address  Is Fraudulent  \
0  30470 Amanda Causeway Suite 446\nLake Ashleyla...              0   
1              0750 Jay Island\nSouth Adam, NC 68936              0   
2         85492 Hill Green\nWest Alexander, TX 53337              0   
3                   PSC 8138, Box 0814\nAPO AA 42879              0   
4                043 Jean Falls\nMeganland, KS 98284              0   

   Account Age Days  Transaction Hour  
0                36                23  
1               158                 2  
2               301                14  
3               118                12  
4               271                 3

In [4]:
df['Quantity'].sample().values

array([4])

In [5]:
def create_event():
    """
    Functie care creeaza un eveniment random din datele citite din fisierul parquet.
    :return: un dictionar cu un singur eveniment
    """
    return {c: df[c].sample().values[0] for c in df.columns}

create_event()

{'Transaction ID': '365e4090-26a3-4dc4-836c-cefee11ed2ce',
 'Customer ID': 'd3ad8f81-47ad-4f13-9947-3679d4e5e763',
 'Transaction Amount': 42.62,
 'Transaction Date': '2024-04-04 14:22:24',
 'Payment Method': 'debit card',
 'Product Category': 'clothing',
 'Quantity': 2,
 'Customer Age': 42,
 'Customer Location': 'Osborneborough',
 'Device Used': 'tablet',
 'IP Address': '32.227.86.253',
 'Shipping Address': '66920 Justin Estates Suite 408\nStephaniestad, NJ 98022',
 'Billing Address': '912 Ruiz Mall\nGreenmouth, IL 96477',
 'Is Fraudulent': 0,
 'Account Age Days': 242,
 'Transaction Hour': 7}

Deoarece datele sunt in format _numpy_, trebuie sa le serializam inainte de a le trimite catre _Kafka_, iar serializarea default a lui json nu stie sa faca aceasta. Vom folosi o clasa custom pentru a serializa datele _numpy_.

In [6]:
import numpy as np
import json

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer, int)):
            return int(obj)
        elif isinstance(obj, (np.floating, float)):
            return float(obj)
        elif isinstance(obj, (np.ndarray, list)):
            return obj.tolist()
        else:
            return super(NumpyEncoder, self).default(obj)

In [7]:
from confluent_kafka import Producer
import time
import uuid
import random

def delivery_report(err, msg):
    """
    Functie de callback care afiseaza daca mesajul a fost trimis cu succes sau nu.
    :param err:  eroarea primita in urma trimiterii mesajului
    :param msg:  mesajul trimis
    """
    if err is not None:
        print(f'Mesajul nu a fost trimis. Eroare: {err}')
    else:
        print(f'Mesajul a fost trimis cu succes. Partitie: {msg.partition()} Offset: {msg.offset()}')
        
def json_serializer(data):
    """
    Functie care serializeaza datele in format json.
    :param data:  datele de serializat
    :return:  datele serializate in format json
    """
    return json.dumps(data,cls=NumpyEncoder).encode('utf-8')

p=Producer({'bootstrap.servers':KAFKA_BOOTSTRAP_SERVER})

def produce():
    """
    Functie care trimite evenimente catre Kafka la intervale random de timp.
    """
    try:
        while True:
            p.produce(TOPIC, key=uuid.uuid4().hex, value=json_serializer(create_event()), callback=delivery_report)
            p.poll(0)
            
            time.sleep(random.randint(1, 5))
    finally:
        p.flush()

In [11]:
produce()

Mesajul a fost trimis cu succes. Partitie: 0 Offset: 32
Mesajul a fost trimis cu succes. Partitie: 0 Offset: 33
Mesajul a fost trimis cu succes. Partitie: 0 Offset: 34
Mesajul a fost trimis cu succes. Partitie: 0 Offset: 35
Mesajul a fost trimis cu succes. Partitie: 0 Offset: 36
Mesajul a fost trimis cu succes. Partitie: 0 Offset: 37
Mesajul a fost trimis cu succes. Partitie: 0 Offset: 38
Mesajul a fost trimis cu succes. Partitie: 0 Offset: 39
Mesajul a fost trimis cu succes. Partitie: 0 Offset: 40
Mesajul a fost trimis cu succes. Partitie: 0 Offset: 41
Mesajul a fost trimis cu succes. Partitie: 0 Offset: 42
Mesajul a fost trimis cu succes. Partitie: 0 Offset: 43
Mesajul a fost trimis cu succes. Partitie: 0 Offset: 44
Mesajul a fost trimis cu succes. Partitie: 0 Offset: 45


KeyboardInterrupt: 